# Data Workflow Lab 1

Clean and summarize Project 3 data.

### Learning Objectives

* Practice text cleaning techniques
* Practice datatype conversion
* Practice filling in missing values with either 0 or the average in the column
* Practice categorical data techniques
* Transform data into usable quantities


In [14]:
% matplotlib inline
import datetime
import numpy as np
import pandas as pd

In [15]:
# Load the data
df = pd.read_csv("/Users/michaelmainzer/Documents/GA/DSI/mike1/01-projects/assets/03-project3-assets/Iowa_Liquor_Sales_sample.csv")
print df.columns

Index([u'Date', u'Store Number', u'City', u'Zip Code', u'County Number',
       u'County', u'Category', u'Category Name', u'Vendor Number',
       u'Item Number', u'Item Description', u'Bottle Volume (ml)',
       u'State Bottle Cost', u'State Bottle Retail', u'Bottles Sold',
       u'Sale (Dollars)', u'Volume Sold (Liters)', u'Volume Sold (Gallons)'],
      dtype='object')


## Clean the data

Let's practice our data cleaning skills on the Project 3 dataset. If you don't remember how to do any of these tasks, look back at your work from the previous weeks or search the internet. There are many blog articles and Stack Overflow posts that cover these topics.

You'll want to complete at least the following tasks:
* Remove redundant columns
* Remove "$" prices from characters and convert values to floats.
* Convert dates to pandas datetime objects
* Convert category floats to integers
* Drop or fill in bad values

In [16]:
# Remove redundant columns
del df['Volume Sold (Gallons)']

In [17]:
# Remove $ from certain columns
df['State Bottle Cost'] = [x.lstrip('$')for x in df['State Bottle Cost']]
df['State Bottle Retail'] = [x.lstrip('$')for x in df['State Bottle Retail']]
df['Sale (Dollars)'] = [x.lstrip('$')for x in df['Sale (Dollars)']]

In [18]:
# Convert dates
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")

In [19]:
# Drop or replace bad values

# Convert integers
df[['State Bottle Cost','State Bottle Retail', 'Sale (Dollars)']] = df[['State Bottle Cost','State Bottle Retail', 
                                                                        'Sale (Dollars)']].astype(float)


In [20]:
df.head()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters)
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.50,6.75,12,81.00,9.0
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,13.75,20.63,2,41.26,1.5
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.0
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,9.50,14.25,6,85.50,10.5
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,7.20,10.80,12,129.60,21.0


## Filter the Data

Some stores may have opened or closed in 2015. These data points will heavily skew our models, so we need to filter them out or find a way to deal with them.

You'll need to provide a summary in your project report about these data points. You may also consider using the monthly sales in your model and including other information (number of months or days each store is open) in your data to handle these unusual cases.

Let's record the first and last sales dates for each store. We'll save this information for later when we fit our models.

## Compute New Columns and Tables

Since we're trying to predict sales and/or profits, we'll want to compute some intermediate data. There are a lot of ways to do thisand good use of pandas is crucial. For example, for each transaction we may want to know:
* margin, retail cost minus bottle cost
* price per bottle
* price per liter

We'll need to make a new dataframe that indexes quantities by store:
* sales per store for all of 2015
* sales per store for Q1 2015
* sales per store for Q1 2016
* total volumes sold
* mean transaction revenue, gross margin, price per bottle, price per liter, etc.
* average sales per day
* number of days open

Make sure to retain other variables that we'll want to use to build our models, such as zip code, county number, city, etc. We recommend that you spend some time thinking about the model you may want to fit and computing enough of the suggested quantities to give you a few options.

Bonus tasks:
* Restrict your attention to stores that were open for all of 2015 and Q1 2016. Stores that opened or closed in 2015 will introduce outliers into your data.
* For each transaction we have the item category. You may be able to determine the store type (primarily wine, liquor, all types of alcohol, etc.) by the most common transaction category for each store. This could be a useful categorical variable for modelling. 

In [21]:
# Margin and Price per liter
df['Margin'] = df['State Bottle Retail'] - df['State Bottle Cost']
df['Price per Liter'] = (df['State Bottle Retail'] / df['Bottle Volume (ml)'])*1000
df.dtypes

Date                    datetime64[ns]
Store Number                     int64
City                            object
Zip Code                        object
County Number                  float64
County                          object
Category                       float64
Category Name                   object
Vendor Number                    int64
Item Number                      int64
Item Description                object
Bottle Volume (ml)               int64
State Bottle Cost              float64
State Bottle Retail            float64
Bottles Sold                     int64
Sale (Dollars)                 float64
Volume Sold (Liters)           float64
Margin                         float64
Price per Liter                float64
dtype: object

In [23]:
# Sales per store, 2015. Only includes stores that were open for all of 2015.

# Filter by our start and end dates
df.sort_values(by=["Store Number", "Date"], inplace=True)
start_date = pd.Timestamp("20150101")
end_date = pd.Timestamp("20151231")
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
sales = df[mask]

# Group by store name
sales = sales.groupby(by=["Store Number"], as_index=False)
# Compute sums, means
sales = sales.agg({"Sale (Dollars)": [np.sum, np.mean],
                   "Volume Sold (Liters)": [np.sum, np.mean],
                   "Margin": np.mean,
                   "Price per Liter": np.mean,
                   "Zip Code": lambda x: x.iloc[0], # just extract once, should be the same
                   "City": lambda x: x.iloc[0],
                   "County Number": lambda x: x.iloc[0]})
# Collapse the column indices
sales.columns = [' '.join(col).strip() for col in sales.columns.values]
# Rename columns

# Quick check
sales.head()

,Store Number,City <lambda>,Sale (Dollars) sum,Sale (Dollars) mean,County Number <lambda>,Price per Liter mean,Zip Code <lambda>,Volume Sold (Liters) sum,Volume Sold (Liters) mean,Margin mean
0,2106,CEDAR FALLS,146326.22,277.658861,7.0,17.856601,50613,9731.85,18.466509,5.166319
1,2113,GOWRIE,9310.22,63.334830,94.0,18.504292,50543,659.85,4.488776,5.445102
2,2130,WATERLOO,111871.43,285.386301,7.0,16.835669,50703,6891.37,17.580026,4.925842
3,2152,ROCKWELL,7721.08,54.759433,17.0,13.020983,50469,633.37,4.491986,4.322624
4,2178,WAUKON,24324.18,102.633671,3.0,16.062136,52172,1917.12,8.089114,4.868861


In [31]:
# Q1 sales, may want to also use aggregate as above to have more columns (means, etc.)

# Sales 2015  Q1
df.sort_values(by=["Store Number", "Date"], inplace=True)
start_date2 = pd.Timestamp("20150101")
end_date2 = pd.Timestamp("20150331")
mask2 = (df['Date'] >= start_date2) & (df['Date'] <= end_date2)
sales2015Q1 = df[mask2]

sales2015Q1 = sales2015Q1.groupby(by=["Store Number"], as_index=False)

sales2015Q1 = sales2015Q1.agg({"Sale (Dollars)": [np.sum, np.mean],
                   "Volume Sold (Liters)": [np.sum, np.mean],
                   "Margin": np.mean,
                   "Price per Liter": np.mean,
                   "Zip Code": lambda x: x.iloc[0], # just extract once, should be the same
                   "City": lambda x: x.iloc[0],
                   "County Number": lambda x: x.iloc[0]})

sales2015Q1.columns = [' '.join(col).strip() for col in sales2015Q1.columns.values]

sales2015Q1.columns = [u'Store Number', u'City', u'Sale (Dollars) sum',
       u'Sale (Dollars) mean', u'County Number',
       u'Price per Liter mean', u'Zip Code',
       u'Volume Sold (Liters) sum', u'Volume Sold (Liters) mean',
       u'Margin mean']

sales2015Q1.head()
# Sales 2016 Q1

Index([u'Store Number', u'City <lambda>', u'Sale (Dollars) sum',
       u'Sale (Dollars) mean', u'County Number <lambda>',
       u'Price per Liter mean', u'Zip Code <lambda>',
       u'Volume Sold (Liters) sum', u'Volume Sold (Liters) mean',
       u'Margin mean'],
      dtype='object')


,Store Number,City,Sale (Dollars) sum,Sale (Dollars) mean,County Number,Price per Liter mean,Zip Code,Volume Sold (Liters) sum,Volume Sold (Liters) mean,Margin mean
0,2106,CEDAR FALLS,39287.29,304.552636,7.0,17.846608,50613,2526.10,19.582171,5.033721
1,2113,GOWRIE,2833.25,67.458333,94.0,19.358141,50543,177.11,4.216905,5.275000
2,2130,WATERLOO,24272.57,278.995057,7.0,17.565430,50703,1447.25,16.635057,5.140920
3,2152,ROCKWELL,2003.46,62.608125,17.0,13.991012,50469,151.74,4.741875,4.836875
4,2178,WAUKON,5856.41,122.008542,3.0,16.724712,52172,409.81,8.537708,4.932083


Proceed with any calculations that you need for your models, such as grouping
sales by zip code, most common vendor number per store, etc. Once you have finished adding columns, be sure to save the dataframe.

In [25]:
# Compute more things
# ...

In [26]:
# Save this dataframe

# Let's add the dates computed above to this data.
#sales["First Date"] = dates['Date amin']
#sales["Last Date"] = dates['Date amax']

#sales.to_csv("sales.csv")